In [12]:
import geopandas as gpd
from shapely.geometry import Polygon, mapping, Point
from shapely import MultiPoint, convex_hull
import fiona

In [5]:
# read google maps data
outline_points = gpd.read_file('PNRM Boundary.csv')

In [6]:
# extract coords from url
outline_points['x'] = outline_points['URL'].apply(lambda x : float(x.split('/')[-1].split(',')[1]))
outline_points['y'] = outline_points['URL'].apply(lambda x : float(x.split('/')[-1].split(',')[0]))


In [7]:
# save to csv
outline_points.to_csv('outline_coords.csv')

In [8]:
# make multipoint from coord
outline_points['geometry'] = outline_points.apply(lambda d : Point(d.x,d.y),axis=1)
mp = MultiPoint([p for p in outline_points['geometry']])

In [9]:
# get convex hull of points
hull = convex_hull(mp)

In [10]:
hull.area

2.621976749963112e-07

In [11]:
# thanks to stackexchange: https://stackoverflow.com/questions/13331698/how-to-apply-a-function-to-two-columns-of-pandas-dataframe
# save convex hull to shapefile for use in arcgis
schema = {
    'geometry': 'Polygon',
    'properties': {'id': 'int'},
}

# Write a new Shapefile
with fiona.open('hull.shp', 'w', 'ESRI Shapefile', schema) as c:
    ## If there are multiple geometries, put the "for" loop here
    c.write({
        'geometry': mapping(hull),
        'properties': {'id': 1},
    })